<a href="https://colab.research.google.com/github/alrz199/LLMs/blob/main/finetuning_FlanT5_lora_from_scractch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook, we will implement Lora from scratch and fine-tune the FlanT5 model with it for text summarization. The code in this notebook is inspired by the following links:

https://github.com/Ryota-Kawamura/Generative-AI-with-LLMs/blob/main/Week-2/Lab_2_fine_tune_generative_ai_model.ipynb

https://lightning.ai/lightning-ai/studios/code-lora-from-scratch

In [ ]:
#installing libraries
!pip install -q transformers dataset evaluate rouge_score loralib peft

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the pac

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install pytorch-lightning accelerate tokenizers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 59.9 MB/s eta 0:00:00


In [ ]:
#import libraries
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

## getting data and the base model

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
model_name='google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## testing the performance of the base model before fine-tuning

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

## tokenizing the data, since the flanT5 model uses -100 for padded ids but tokenizer uses 0, we should change pdded places to -100 to use the data in our model.

In [ ]:
def tokenize_function(example):
#incorporating prompt engineering
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = start_prompt + example["dialogue"] + end_prompt
#get the input_ids, attention_mask, and labels(desired outputs)
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['input_mask'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").attention_mask
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'][example['labels'] == tokenizer.pad_token_id] = -100
    return example

tokenized_datasets = dataset.map(tokenize_function)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue','summary'])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'input_mask', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'input_mask', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'input_mask', 'labels'],
        num_rows: 1500
    })
})


In [ ]:
tokenized_datasets.set_format(type="torch")

In [ ]:
tokenized_datasets['train'][0]

{'input_ids': tensor([[12198,  1635,  1737,     8,   826,  3634,     5,  1713,   345, 13515,
            536,  4663,    10,  2018,     6,  1363,     5,  3931,     5,    27,
             31,    51,  7582, 12833,    77,     7,     5,  1615,    33,    25,
            270,   469,    58,  1713,   345, 13515,   357,  4663,    10,    27,
            435,    34,   133,    36,     3,     9,   207,   800,    12,   129,
              3,     9,   691,    18,   413,     5,  1713,   345, 13515,   536,
           4663,    10,  2163,     6,   168,     6,    25,    43,    29,    31,
             17,   141,    80,    21,   305,   203,     5,   148,   225,    43,
             80,   334,   215,     5,  1713,   345, 13515,   357,  4663,    10,
             27,   214,     5,    27,  2320,    38,   307,    38,   132,    19,
           1327,  1786,     6,   572,   281,   217,     8,  2472,    58,  1713,
            345, 13515,   536,  4663,    10,  1548,     6,     8,   200,   194,
             12,  1792,  22

## Let's check the original model; we will add lora to its attention layers

In [ ]:
print(original_model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
#determining the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## defining lora layer and checking its trainable params

In [ ]:
from torch import nn
class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        weights_A = torch.Tensor(in_dim, rank).bfloat16()
        weights_B = torch.Tensor(rank,out_dim).bfloat16()
        std_dev = 1 / torch.sqrt(torch.tensor(rank))
        self.A = nn.Parameter(weights_A)
        self.B = nn.Parameter(weights_B)
        with torch.no_grad():
          self.A = nn.Parameter((torch.randn(in_dim, rank) * std_dev).bfloat16())
          self.B = nn.Parameter((torch.zeros(rank, out_dim)).bfloat16())
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x

In [ ]:
for p,v in LoRALayer(5,8,8,16).named_parameters():
  print(p)

A
B


## linearwithLora in the next cell gets a linear layer(one part of the attention layer) and adds lora layer to it

In [ ]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )
    def forward(self, x):
        return self.linear(x) + self.lora(x)

#next, we will ensure that all parameters from the original model are frozen.

In [ ]:
for param in original_model.parameters():
    param.requires_grad = False

#adding lora to all attention layers

In [ ]:
from functools import partial

# default hyperparameter choices
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05

assign_lora = partial(LinearWithLoRA, rank=lora_r, alpha=lora_alpha)
for i in range(12):
  original_model.encoder.block[i].layer[0].SelfAttention.q = assign_lora(original_model.encoder.block[i].layer[0].SelfAttention.q)
  original_model.encoder.block[i].layer[0].SelfAttention.k = assign_lora(original_model.encoder.block[i].layer[0].SelfAttention.k)
  original_model.encoder.block[i].layer[0].SelfAttention.v = assign_lora(original_model.encoder.block[i].layer[0].SelfAttention.v)
  original_model.encoder.block[i].layer[0].SelfAttention.o = assign_lora(original_model.encoder.block[i].layer[0].SelfAttention.o)

  original_model.decoder.block[i].layer[0].SelfAttention.q = assign_lora(original_model.decoder.block[i].layer[0].SelfAttention.q)
  original_model.decoder.block[i].layer[0].SelfAttention.k = assign_lora(original_model.decoder.block[i].layer[0].SelfAttention.k)
  original_model.decoder.block[i].layer[0].SelfAttention.v = assign_lora(original_model.decoder.block[i].layer[0].SelfAttention.v)
  original_model.decoder.block[i].layer[0].SelfAttention.o = assign_lora(original_model.decoder.block[i].layer[0].SelfAttention.o)

  original_model.decoder.block[i].layer[1].EncDecAttention.q = assign_lora(original_model.decoder.block[i].layer[1].EncDecAttention.q)
  original_model.decoder.block[i].layer[1].EncDecAttention.k = assign_lora(original_model.decoder.block[i].layer[1].EncDecAttention.k)
  original_model.decoder.block[i].layer[1].EncDecAttention.v = assign_lora(original_model.decoder.block[i].layer[1].EncDecAttention.v)
  original_model.decoder.block[i].layer[1].EncDecAttention.o = assign_lora(original_model.decoder.block[i].layer[1].EncDecAttention.o)

In [ ]:
for p,value in original_model.encoder.block[0].layer[0].SelfAttention.q.named_parameters():
  print(p)

linear.weight
lora.A
lora.B


#let's check the number of trainable params when we use lora

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for name, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 1769472
all model parameters: 249347328
percentage of trainable model parameters: 0.71%


#defining the train and test dataloader, we will use batch_size=2 but in trainer we will use "accumulate_grad_batches", to save memory

In [ ]:
train_data_loader = torch.utils.data.DataLoader(tokenized_datasets['train'], batch_size=2, shuffle=True, pin_memory=True)
val_data_loader = torch.utils.data.DataLoader(tokenized_datasets['validation'], batch_size=2, shuffle=False, pin_memory=True)

#defining and training the model; the flant5 model gets labels and returns the loss as one of its outputs. We can also work with model(...).logits, which returns the output probability vectors

In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts , ReduceLROnPlateau


class TextSummarization(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = original_model
        self.param_train = {}
        for param,value in self.model.named_parameters():
            if value.requires_grad:
                self.param_train[param] = value
    def forward(self, ids,attention_masks,targets):
        outputs = self.model(input_ids = ids.squeeze(),
                             attention_mask = attention_masks.squeeze(),
                             labels = targets.squeeze())
        return outputs
    def training_step(self,batch,batch_idx):
        out = self.forward(batch['input_ids'], batch['input_mask'],batch['labels'])
        loss = out['loss']
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self,batch,batch_idx):
        out = self.forward(batch['input_ids'], batch['input_mask'],batch['labels'])
        loss = out['loss']
        self.log("val_loss", loss, prog_bar=True)
        return loss
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(list(self.param_train.values()), lr=1e-4,maximize=False)
        return {
        "optimizer": optimizer,
        # "lr_scheduler": {
        # "scheduler":  CosineAnnealingWarmRestarts(optimizer, T_0=1000, T_mult=2)},
        # "gradient_clip_val": 1.0
}

In [ ]:
# Initialize a trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
mymodel=TextSummarization()
trainer = Trainer(max_epochs=2,accelerator='gpu', num_nodes = 1 ,accumulate_grad_batches=64)
# Train the model ⚡
trainer.fit(mymodel, train_data_loader,val_dataloaders=val_data_loader)
print('Finished Training')

# let's check the model's performance again

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    mymodel.model.generate(
        inputs["input_ids"].to('cuda'),
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------